In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama

In [1]:
from langchain.document_loaders import TextLoader

# Load PDF files
raw_documents_1 = TextLoader("./Herbs that synergize with TaxanesKadcylaEnhertu v7.pdf").load()
raw_documents_2 = TextLoader("./Herbs that synergize with Carboplatin and Cisplatin V3.pdf").load()
raw_documents_3 = TextLoader("./McKinney Surgery Pre and Post Op protocol.pdf").load()

# Combine all documents into one list
all_raw_documents = raw_documents_1 + raw_documents_2 + raw_documents_3


RuntimeError: Error loading ./Herbs that synergize with TaxanesKadcylaEnhertu v7.pdf

In [3]:
from langchain.document_loaders import PyPDFLoader

# Load PDF files
loader_1 = PyPDFLoader("./Herbs that synergize with TaxanesKadcylaEnhertu v7.pdf")
loader_2 = PyPDFLoader("./Herbs that synergize with Carboplatin and Cisplatin V3.pdf")
loader_3 = PyPDFLoader("./McKinney Surgery Pre and Post Op protocol.pdf")

# Load the documents
raw_documents_1 = loader_1.load()
raw_documents_2 = loader_2.load()
raw_documents_3 = loader_3.load()

# Combine all documents into one list
all_raw_documents = raw_documents_1 + raw_documents_2 + raw_documents_3


Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define chunking parameters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documents = text_splitter.split_documents(all_raw_documents)

print(len(documents))  # Check the number of chunks
print(documents[0])  # Preview the first document chunk


108
page_content='Herbs and Natural Substances Synergies with Taxanes  (Taxol, Paclitaxel, Docetaxel, Taxotere), Vinorelbine, Enhertu, and Kadcyla  
 Disclaimer: This document is for informational purposes, does not constitute medical advice, and should not replace the advice of a medical professional.' metadata={'producer': 'macOS Version 10.15.7 (Build 19H2026) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20230827223908Z00'00'", 'title': 'Microsoft Word - Herbs that synergize with TaxanesKadcylaEnhertu v7.docx', 'moddate': "D:20230827223908Z00'00'", 'source': './Herbs that synergize with TaxanesKadcylaEnhertu v7.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}


In [6]:
from langchain.embeddings import OllamaEmbeddings

# Initialize Ollama embeddings
embed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

# Create embeddings for the document chunks
embeddings = embed.embed_documents(documents)


In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings

# Initialize the FAISS vector store (or another vector store you prefer)
vector_store = FAISS.from_documents(documents, embed)

# Save vector store for future use
vector_store.save("my_vector_store.index")


AttributeError: 'FAISS' object has no attribute 'save'

In [12]:
import faiss
# Save the FAISS index to a file (use the faiss.write_index method)
faiss.write_index(vector_store.index, "my_vector_store.index")

In [13]:
query = "Summarize the article about Carboplatin usage in cancer treatment."

# Retrieve the relevant document chunks
retrieved_docs = vector_store.similarity_search(query)
print(retrieved_docs)


[Document(id='8ad27fcf-2910-4800-8a8a-3d842759cacf', metadata={'producer': 'macOS Version 10.15.7 (Build 19H2026) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20230721233629Z00'00'", 'title': 'Microsoft Word - Herbs that synergize with Carboplatin and Cisplatin V3.docx', 'moddate': "D:20230721233629Z00'00'", 'source': './Herbs that synergize with Carboplatin and Cisplatin V3.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='a.\t\t\tCarboplatin\tis\tless\ttoxic\tto\tthe\tkidneys,\tbut\tmore\tmutagenic\tand\tdamaging\tto\tthe\tbone\tmarrow\tand\tblood\tcell\tcounts\tthan\tcisplatin.\tCarboplatin\tproduces\telectrolyte\t(blood\tminerals)\timbalances,\tnausea\tand\tvomiting,\tabnormal\tliver\tfunction,\tnerve\tdamage,\tand\tmuscle\tpain.\tConsider\textra\tglutamine'), Document(id='cade82d6-b2ff-45b3-9f5e-46185195a6f9', metadata={'producer': 'macOS Version 10.15.7 (Build 19H2026) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20230721233629Z00'00'"

In [14]:
from langchain.chains import RetrievalQA
from langchain.llms import Ollama

# Initialize the LLM with Ollama or another language model
llm = Ollama(model="monic-chat")

# Use the retrieval-augmented chain to generate the summary
qa_chain = RetrievalQA.from_chain_type(llm, vector_store)

# Generate summary for the user input
response = qa_chain.run(query)
print(response)


C:\Users\premm\AppData\Local\Temp\ipykernel_14780\3243159365.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="monic-chat")


ValueError: Got unsupported chain type: <langchain_community.vectorstores.faiss.FAISS object at 0x000002397C70EAB0>. Should be one of dict_keys(['stuff', 'map_reduce', 'refine', 'map_rerank'])